In [ ]:
import tempfile
import pyaudio
import openai
import io

from pydub import AudioSegment

# Initialize PyAudio
chunk_size = 1024
audio_format = pyaudio.paInt16
channels = 1
sample_rate = 16000
p = pyaudio.PyAudio()

# Initialize frames to store audio
audio_frames = []

In [ ]:
# Create the audio stream and transcription loop
stream = p.open(format=audio_format,
                channels=channels,
                rate=sample_rate,
                input=True,
                frames_per_buffer=chunk_size)

# Collect the audio until stopped
while True:
  try:
    # Read audio data from the stream
    audio_data = stream.read(chunk_size)
    audio_frames.append(audio_data)

  except KeyboardInterrupt:
    # Stop the stream, close it, and terminate the PyAudio instance
    stream.stop_stream()
    stream.close()
    p.terminate()
    break

In [ ]:
# Loop through all the sound chunks and create a single audio file
# sound_chunk = pydub.AudioSegment.empty()
# for audio_frame in audio_frames:
#   sound = pydub.AudioSegment(data=audio_frame)
#   sound_chunk += sound


# Store the audio in a temporary file
# with tempfile.NamedTemporaryFile(suffix='.wav', delete=True) as tmp:
wav_data = io.BytesIO(b"".join(audio_frames))
audio_segment = AudioSegment.from_wav(wav_data)

audio_segment.export('test.wav', format="wav")
# response = openai.Audio.transcribe("whisper-1", tmp)
print(len(audio_segment))


In [ ]:
import pyaudio
from datetime import datetime
from pydub import AudioSegment

# Set the parameters for the audio recording
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
CHUNK = 1024
RECORD_SECONDS = 5

# Initialize the PyAudio object
audio = pyaudio.PyAudio()

# Open a new stream for recording
stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE,
                    input=True, frames_per_buffer=CHUNK)

# Start recording
frames = []
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

# Stop recording
stream.stop_stream()
stream.close()
audio.terminate()

# Create a new AudioSegment from the recorded audio frames
audio_segment = AudioSegment(
    b''.join(frames),
    frame_rate=RATE,
    sample_width=audio.get_sample_size(FORMAT),
    channels=CHANNELS
)

# Get the current date and time for the file name
now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Save the audio as a WAV file
audio_file = f"{now}.wav"
audio_segment.export(audio_file, format="wav")